In [6]:
import yfinance as yf
import pandas as pd
import math 

In [7]:
symbols = pd.read_csv("../data/simple_tickers.csv")["Ticker"].tolist()
owned_tickers = pd.read_csv("../data/owned_tickers.csv")["Ticker"].tolist()

In [ ]:
df = pd.DataFrame(columns=[
    'Ticker', 
    'short_name', 
    'recommendation_score',
    'value_score',  
    'analyst_mean%', 
    'insider_buy%', 
    'sector', 
    'industry', 
    'country',
    'P/E (TTM)', 
    'ROA%',
    'in_portfolio'
])
for symbol in symbols: 
    ticker = yf.Ticker(symbol)
    try:
        if symbol in owned_tickers:
            in_portfolio = True
        else:
            in_portfolio = False
        analyst_mean = 0
        if 'mean' in ticker.analyst_price_targets and ticker.info['currentPrice']:
            analyst_mean = ticker.analyst_price_targets['mean'] / ticker.info['currentPrice'] - 1
        roa = ticker.info['returnOnAssets']*100
        pe = ticker.info['trailingPE']
        value_score = format(2.6 + math.log(roa+5) - math.log(pe+25), '.2f')
        insider_buy = 0
        if 'Shares' in ticker.insider_purchases.columns:
            insider_buy = format(ticker.insider_purchases.loc[ticker.insider_purchases.index[4], 'Shares'], '.2f')
            if insider_buy == '<NA>':
                insider_buy = 0
        recommendation_score = format(float(value_score) + analyst_mean * 1.25 + float(insider_buy) * 2.0, '.2f')
        df.loc[symbol] = [
            symbol, 
            ticker.info['shortName'], 
            recommendation_score,
            value_score,
            format(analyst_mean * 100, '.2f'), 
            insider_buy,
            ticker.info['sector'], 
            ticker.info['industry'], 
            ticker.info['country'],
            format(pe, '.2f') if isinstance(pe, (int, float)) else 'N/A', 
            format(roa, '.2f') if isinstance(roa, (int, float)) else 'N/A',
            in_portfolio
        ]
    except Exception as e:
        print(f"Error with {symbol}: {e}. Remove ticker from list.")
df.to_csv('screener.csv', index=False)
display(df)
print(f'Got data for {len(df)} stocks')

Error with GOGO: 'trailingPE'. Removed Ticker from list.
Error with PARRO.PA: 'trailingPE'. Removed Ticker from list.


,Ticker,short_name,recommendation_score,value_score,analyst_mean%,insider_buy%,sector,industry,country,P/E (TTM),ROA%,in_portfolio
MATAS.CO,MATAS.CO,Matas A/S,1.44,1.03,32.55,0,Consumer Cyclical,Specialty Retail,Denmark,18.40,4.06,False
TRIFOR.CO,TRIFOR.CO,Trifork Group AG,1.07,0.81,20.77,0,Technology,Information Technology Services,Switzerland,16.60,1.92,False
RNMBY,RNMBY,Rheinmetall AG,0.30,0.30,0.00,0,Industrials,Aerospace & Defense,Germany,95.26,7.06,True
SAABF,SAABF,Saab AB,0.43,0.43,0.00,0,Industrials,Aerospace & Defense,Sweden,52.08,3.80,True
BCKIY,BCKIY,Babcock International Group Plc,0.94,0.94,0.00,0,Industrials,Engineering & Construction,United Kingdom,27.82,5.08,False
BAESY,BAESY,BAE Systems PLC,1.06,0.84,17.94,0,Industrials,Aerospace & Defense,United Kingdom,29.78,4.40,False
IVSO.ST,IVSO.ST,INVISIO AB,1.66,1.49,13.99,0,Technology,Communication Equipment,Sweden,49.30,19.45,False
NSKFF,NSKFF,Kongsberg Gruppen ASA,0.74,0.74,0.00,0,Industrials,Aerospace & Defense,Norway,57.14,7.82,True
GMAB,GMAB,Genmab A/S,2.11,1.75,28.57,0,Healthcare,Biotechnology,Denmark,13.58,11.54,False
GN.CO,GN.CO,GN Store Nord A/S,1.54,0.99,44.23,0,Healthcare,Medical Devices,Denmark,17.63,3.50,False


Got data for 43 stocks


### Single import

In [10]:
ticker = "GOGO"
print(f"P/E: {yf.Ticker(ticker).info['trailingPE']:.2f}")
print(f"ROA%: {yf.Ticker(ticker).info['returnOnAssets']*100:.2f}")

Failed to perform, curl: (16) . See https://curl.se/libcurl/c/libcurl-errors.html first for more details.


TypeError: argument of type 'NoneType' is not iterable